In [33]:
import io
import fitz
import re
from PIL import Image, ImageDraw, ImageFont, ImageColor
import operator
import pandas as pd 
import numpy as np
from datetime import datetime
import math
from tqdm import tqdm

# Ploting funcs

In [34]:
def plot_genus_blocks(page_df, draw, color = '#6c899e', w = 3):
    try:
        genus_list = page_df['draw_genus'].unique()
    except:
        #print("no GENUS found")
        return 

    for g in genus_list:
        temp_df = page_df[(page_df['draw_genus'] == g)]
        g_x0 = temp_df['x0'].min()
        g_y0 = temp_df['y0'].min()
        g_x1 = temp_df['x1'].max()
        g_y1 = temp_df['y1'].max()

        draw.rectangle((g_x0, g_y0, g_x1, g_y1), fill=None, outline=ImageColor.getrgb(color), width = w)
        
def plot_epithet_blocks(page_df, draw, color = '#660066', w = 3):
    try:
        epithet_list = page_df['draw_epithet'].unique()
    except:
        print("no EPITHET found")
        return 
    
    for e in epithet_list:
        temp_df = page_df[(page_df['draw_epithet'] == e)]
        e_x0 = temp_df['x0'].min()
        e_y0 = temp_df['y0'].min()
        e_x1 = temp_df['x1'].max()
        e_y1 = temp_df['y1'].max()

        draw.rectangle((e_x0, e_y0, e_x1, e_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_author_blocks(page_df, draw, color = '#a3a3a3', w = 1):
    try:
        author_list = page_df['draw_author'].unique()
    except:
        print("no AUTHOR found")
        return 

    for a in author_list:
        temp_df = page_df[(page_df['draw_author'] == a)]
        e_x0 = temp_df['x0'].min()
        e_y0 = temp_df['y0'].min()
        e_x1 = temp_df['x1'].max()
        e_y1 = temp_df['y1'].max()

        draw.rectangle((e_x0, e_y0, e_x1, e_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_infra_blocks(page_df, draw, color = '#ff6289', w = 1):
    try:
        infra_list = page_df['draw_infra'].unique()
    except:
        print("no INFRA Spp. found")
        return 

    for infra_spp in infra_list:
        temp_df = page_df[(page_df['draw_infra'] == infra_spp)]
        e_x0 = temp_df['x0'].min()
        e_y0 = temp_df['y0'].min()
        e_x1 = temp_df['x1'].max()
        e_y1 = temp_df['y1'].max()

        draw.rectangle((e_x0, e_y0, e_x1, e_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_valid_words(page_df, draw, color = '#660044', w = 2):
    blocks = page_df['block_no'].unique()
    """for b in blocks:
        lines = page_df[page_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            cond = (page_df['line_no'] == l) & (page_df['block_no'] == b)
            words = page_df[cond]['word_no'].unique()
            page_df = page_df.copy()
            for w in words:
                x0 = page_df[(cond) & (page_df['word_no'] == w)]['x0'].item()
                y0 = page_df[(cond) & (page_df['word_no'] == w)]['y0'].item()
                x1 = page_df[(cond) & (page_df['word_no'] == w)]['x1'].item()
                y1 = page_df[(cond) & (page_df['word_no'] == w)]['y1'].item()
                draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb(color), width = w)
    """
    for index, row in page_df.iterrows():
        x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1'] 
        draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb(color), width = w)

# Import Vol1 Index

In [35]:
#pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf"
"""pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf"
index = range(616, 639)
doc = fitz.open(pdf_dir)
pages = [doc[i] for i in range(doc.page_count)] #doesn't work anymore? [doc[i] for i in range(doc.pageCount)]
#index = list(range(555, 583))

pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf"
index = range(616, 639)"""
pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf"
doc = fitz.open(pdf_dir)
pages = [doc[i] for i in range(doc.page_count)]
index = list(range(703, 725))


TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

indent_groups = []
indent_err = 15

# regex based boolean functions

In [24]:
def valid(word):
    """
    valid words are words that are:
    - at least 2 characters
        - unless it's x (symbol for hybrid)
    """
    return (not bool(re.search(r"[0-9]+[,.]?", word))) and \
            (word != 'NOUVELLE' and word != 'FLORE') and \
            (word != 'INDEX' and word != 'SPECIERUM') and \
            (len(word) > 1 or \
                word == 'x' or word == 'X' or word == '×' or word == r'\u00D7') and \
            ''.join(e for e in word if e.isalpha()).isalpha()
    
def is_genus(word):
    """
    A word in the index might be a genus if it satisfies the following properties:
    - letters: french alphabet + at most one hyphen (which is not first or last letter)
        - first letter upper case
        - all but first lowecase 
    in regex: ^[A-ZÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]{1}[a-zàâäèéêëîïôœùûüÿç]*[-]?[a-zàâäèéêëîïôœùûüÿç]+$ #ignoring strict beggining and end cause of noise
        * based on the current expression it'd also be at least 2 letters long
    """
    regex = r"[A-ZÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ\u00D7]{1}[a-zàâäèéêëîïôœùûüÿç]*[-]?[a-zàâäèéêëîïôœùûüÿç]+"
    return re.search(regex, word)
    

def is_epithet(word):
    """
    A word in the index might be an epithet if it satisfies the following properties:
    - letters: french alphabet + at most one hyphen (which is not first or last letter)
        - all letters lowecase 
    in regex: ^[a-zàâäèéêëîïôœùûüÿç]+[-]?[a-zàâäèéêëîïôœùûüÿç]+$ #ignoring strict beggining and end cause of noise 
        * based on the current expression it'd also be at least 2 letters long
    """
    regex = r"[a-zàâäèéêëîïôœùûüÿç\u00D7]+[-]?[a-zàâäèéêëîïôœùûüÿç]+"
    return re.search(regex, word)
    
def is_hybrid(word):
    regex = r"^(([Xx\u00D7])|([Xx\u00D7]\.))$"
    return re.search(regex, word)

def is_infra(word):
    regex = r"^(var\.)|(subsp\.)"
    return re.search(regex, word)

# pre-processing func

In [25]:
def preprocessing(page_num, indent_err = 30):
    
    #initiate dataframe
    page_df = pd.DataFrame(pages[page_num].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
    
    #to get blocks, lines, span etc
    """blocks = page.get_text("dict")["blocks"]
    for b in blocks:
        if "lines" in b:
            for l in b["lines"]:
                for span in b["lines"]:
                    for word in span["spans"][0]["text"].split():
                        print("word:", word, "\t\t\t flags:", span["spans"][0]["flags"], "\t\t\t font:", span["spans"][0]["font"])"""

    #idea: get this and get coordinates, words, block_no_etc based on this.
    #or perhaps create a db from the dictionary of page.get_text("dict")and the create rows for each text line by splitting it:
    #based on this: https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows

    #add page number to dataframe
    page_df['page_num'] = np.array([page_num]*page_df.shape[0])
    #initiate all columns that will be added
    page_df['genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['taxon rank'] = np.array([np.NaN]*page_df.shape[0])
    page_df['error_check'] = np.array([np.NaN]*page_df.shape[0])
    page_df['flags'] = np.array([np.NaN]*page_df.shape[0])
    page_df['font'] = np.array([np.NaN]*page_df.shape[0])
    #updating coordinates to represent target DPI
    page_df['x0'], page_df['y0'], page_df['x1'], page_df['y1'] = page_df['in_x0']*TARGET_DPI/ 72, page_df['in_y0']*TARGET_DPI/ 72, page_df['in_x1']*TARGET_DPI/ 72, page_df['in_y1']*TARGET_DPI/ 72
    #get x corner coordinates 
    x_min = page_df['x0'].min()
    x_max = page_df['x1'].max()

    y_max = page_df['y1'].max()

    #Remove the extra flore - 18 at page 545
    if page_num == index[4]:
        page_df = page_df[~((page_df["word"] == 'Flore') & (page_df['y1'] == y_max))]
    #invalid words dataframe -- for error checking
    pruned_words_df = page_df[~page_df["word"].apply(valid)].reset_index()
    #prune out invalid words (based on function valid)
    #page_df = page_df[page_df["word"].apply(valid)].reset_index()
    
    indent_groups = []
    blocks = page_df['block_no'].unique()
    for b in blocks:
        lines = page_df[page_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            #reset word_no values (useful for cases where word that was originally at 0th index was pruned out)
            cond = (page_df['line_no'] == l) & (page_df['block_no'] == b)
            num_words = len(page_df[cond]['word_no'])
            page_df.loc[cond, 'word_no'] = np.arange(num_words).astype(int) #this is slowww
            #set column number (0 or 1)
            x_0 = page_df[cond]['x0'].min()
            #THIS DOESN'T WORK AAAA -- issue was with line no thing
            if not np.isnan(x_0):
                page_df.loc[cond, 'col_no'] = np.array([int(x_0 > ((x_min + x_max) / 2))]*num_words).astype(int)

                #initiate indent groups -- only first word should get an indent_group value 
                new_group = True
                for g_i in range(len(indent_groups)):
                    g = indent_groups[g_i]
                    g_arr = np.array(g)
                    if x_0 <= np.mean(g_arr) + indent_err and x_0 >= np.mean(g_arr) - indent_err:
                        g.append(x_0)
                        new_group = False
                        page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)
                if new_group:
                    indent_groups.append([x_0])
                    g_i = len(indent_groups) - 1
                    page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)

    #print("indent groups:", indent_groups)
    #return updated page_df, pruned_words_df, indent groups
    return page_df.reset_index(), pruned_words_df, indent_groups

#https://stackoverflow.com/questions/53468558/adding-image-to-pandas-dataframe

In [36]:
def preprocessing(page_num, indent_err = 30):
    
    #initiate dataframe
    page_df = pd.DataFrame(pages[page_num].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
    
    #to get blocks, lines, span etc
    """blocks = page.get_text("dict")["blocks"]
    for b in blocks:
        if "lines" in b:
            for l in b["lines"]:
                for span in b["lines"]:
                    for word in span["spans"][0]["text"].split():
                        print("word:", word, "\t\t\t flags:", span["spans"][0]["flags"], "\t\t\t font:", span["spans"][0]["font"])"""

    #idea: get this and get coordinates, words, block_no_etc based on this.
    #or perhaps create a db from the dictionary of page.get_text("dict")and the create rows for each text line by splitting it:
    #based on this: https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows

    #add page number to dataframe
    page_df['page_num'] = np.array([page_num]*page_df.shape[0])
    #initiate all columns that will be added
    page_df['genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['taxon rank'] = np.array([np.NaN]*page_df.shape[0])
    page_df['error_check'] = np.array([np.NaN]*page_df.shape[0])
    page_df['flags'] = np.array([np.NaN]*page_df.shape[0])
    page_df['font'] = np.array([np.NaN]*page_df.shape[0])
    #updating coordinates to represent target DPI
    page_df['x0'], page_df['y0'], page_df['x1'], page_df['y1'] = page_df['in_x0']*TARGET_DPI/ 72, page_df['in_y0']*TARGET_DPI/ 72, page_df['in_x1']*TARGET_DPI/ 72, page_df['in_y1']*TARGET_DPI/ 72
    #get x corner coordinates 
    x_min = page_df['x0'].min()
    x_max = page_df['x1'].max()

    y_max = page_df['y1'].max()

    #Remove the extra flore - 18 at page 545
    if page_num == index[4]:
        page_df = page_df[~((page_df["word"] == 'Flore') & (page_df['y1'] == y_max))]
    #invalid words dataframe -- for error checking
    pruned_words_df = page_df[~page_df["word"].apply(valid)].reset_index()
    #prune out invalid words (based on function valid)
    #page_df = page_df[page_df["word"].apply(valid)].reset_index()
    
    indent_groups = []
    blocks = page_df['block_no'].unique()
    for b in blocks:
        lines = page_df[page_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            #reset word_no values (useful for cases where word that was originally at 0th index was pruned out)
            cond = (page_df['line_no'] == l) & (page_df['block_no'] == b)
            num_words = len(page_df[cond]['word_no'])
            page_df.loc[cond, 'word_no'] = np.arange(num_words).astype(int) #this is slowww
            #set column number (0 or 1)
            x_0 = page_df[cond]['x0'].min()
            #THIS DOESN'T WORK AAAA -- issue was with line no thing
            if not np.isnan(x_0):
                page_df.loc[cond, 'col_no'] = np.array([int(x_0 > ((x_min + x_max) / 2))]*num_words).astype(int)

                #initiate indent groups -- only first word should get an indent_group value 
                new_group = True
                for g_i in range(len(indent_groups)):
                    g = indent_groups[g_i]
                    g_arr = np.array(g)
                    if x_0 <= np.mean(g_arr) + indent_err and x_0 >= np.mean(g_arr) - indent_err:
                        g.append(x_0)
                        new_group = False
                        page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)
                if new_group:
                    indent_groups.append([x_0])
                    g_i = len(indent_groups) - 1
                    page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)

    #print("indent groups:", indent_groups)
    #return updated page_df, pruned_words_df, indent groups
    return page_df.reset_index(), pruned_words_df, indent_groups

#https://stackoverflow.com/questions/53468558/adding-image-to-pandas-dataframe

In [37]:
page_df = pd.DataFrame(pages[page_num-10].get_text("dict")['blocks'])
page_df


,number,type,bbox,width,height,ext,colorspace,xres,yres,bpc,transform,size,image,lines
0,0,1,"(179.88299560546875, 31.599985122680664, 225.0...",188.0,24.0,png,1.0,96.0,96.0,1.0,"(45.119998931884766, 0.0, -0.0, 5.760000228881...",1633.0,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,NaN
1,1,0,"(180.0, 27.748294830322266, 408.8352966308594,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'spans': [{'size': 8.949877738952637, 'flags..."
2,2,1,"(219.06500244140625, 53.92002868652344, 248.82...",124.0,33.0,png,1.0,96.0,96.0,1.0,"(29.760000228881836, 0.0, -0.0, 7.920000076293...",1633.0,"b""\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...",NaN
3,3,0,"(47.040000915527344, 50.01697540283203, 409.14...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'spans': [{'size': 10.94486141204834, 'flags..."
4,4,1,"(54.06999969482422, 66.64000701904297, 61.9899...",33.0,26.0,png,1.0,96.0,96.0,1.0,"(7.920000076293945, 0.0, -0.0, 6.2399997711181...",1633.0,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,65,0,"(22.799999237060547, 549.2169799804688, 407.42...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'spans': [{'size': 10.94486141204834, 'flags..."
66,66,1,"(68.6500015258789, 565.1199951171875, 125.5299...",237.0,43.0,png,1.0,96.0,96.0,1.0,"(56.880001068115234, 0.0, -0.0, 10.31999969482...",1633.0,"b""\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...",NaN
67,67,0,"(22.799999237060547, 560.9769897460938, 407.35...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'spans': [{'size': 10.94486141204834, 'flags..."
68,68,1,"(53.32699966430664, 588.4000244140625, 73.9669...",86.0,33.0,png,1.0,96.0,96.0,1.0,"(20.639999389648438, 0.0, -0.0, 7.920000076293...",1633.0,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,NaN


In [38]:
line_dicts = page_df[~page_df['lines'].isnull()].explode('lines', ignore_index=False)
list(line_dicts['lines'])
"""
pd.DataFrame.from_dict(
    {(i, j): user_dict[i][j] for i in user_dict.keys() for j in user_dict[i].keys()},
    orient="index",
)""";
df = pd.DataFrame(list((pd.DataFrame(list(line_dicts['lines']))['spans'].explode('spans'))))
df['text'] = df['text'].apply(lambda x : list(x.split()))
df = df.explode('text')
#df['word_no'] = df['text'].apply(lambda x : x[0])
#df['word'] = df['text'].apply(lambda x : x[1])
#df[df['word'].apply(lambda x : x.isalpha())]

split_df = pd.DataFrame(df['bbox'].tolist(), columns=['x0', 'y0', 'x1', 'y1'])
# concat df and split_df
df = pd.concat([df.reset_index(), split_df], axis=1)

#df['bbox']
#[['flags', 'font']]
#Ideas to fix the rest of this: 
#get line number and block number similarly
#okay the numbering seems to get messed up this way yikes? okay need to add position on spans and that should fix it!d

page_df_old = pd.DataFrame(pages[page_num-10].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
page_df_old.head()
df.head()

,index,size,flags,font,color,ascender,descender,text,origin,bbox,x0,y0,x1,y1
0,0,8.949878,4,Times-Roman,0,1.053,-0.281,NOUVELLE,"(180.0, 37.1199951171875)","(180.0, 27.748294830322266, 225.6593017578125,...",180.000000,27.748295,225.659302,39.620895
1,1,8.855388,4,Times-Roman,0,1.053,-0.281,FLORE,"(225.6593017578125, 37.1199951171875)","(225.6593017578125, 27.748294830322266, 260.25...",225.659302,27.748295,260.255157,39.620895
2,2,8.855388,4,Times-Roman,0,1.053,-0.281,691,"(394.1575927734375, 37.1199951171875)","(394.1575927734375, 27.748294830322266, 408.83...",394.157593,27.748295,408.835297,39.620895
3,3,10.944861,4,Times-Roman,0,1.053,-0.281,En,"(47.040000915527344, 61.5999755859375)","(47.040000915527344, 50.01697540283203, 214.55...",47.040001,50.016975,214.555969,64.690979
4,3,10.944861,4,Times-Roman,0,1.053,-0.281,postulant,"(47.040000915527344, 61.5999755859375)","(47.040000915527344, 50.01697540283203, 214.55...",47.040001,50.016975,214.555969,64.690979


In [39]:
joined_df = pd.concat([df, page_df_old], axis=1, join="outer")
assert joined_df[joined_df['text'] != joined_df['word']].empty
joined_df = joined_df[['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no', 'flags', 'font', 'size', 'color']]

In [40]:
df.head()
(df['in_x0'] <= page_df_old['in_x0']) & (page_df_old['in_x0'] <= df['in_x1']) & \
    (df['in_y0'] <= page_df_old['in_y0']) & (page_df_old['in_y0'] <= df['in_y1'])

KeyError: 'in_x0'

In [41]:
a = preprocessing(2, indent_err = 30)
pages[page_num]

import json
page = doc.load_page(page_num)
"""blocks = json.loads(page)["blocks"]
for block in blocks:
    if "lines" in block:
        for line in block["lines"]:
            line_text = ""
            for span in block["lines"]:
                line_text += span["spans"][0]["text"].encode("utf-8")
            if last_text != line_text:
                is_recording, main_class = add_line(line_text, is_recording, main_class)
                last_text = line_text""";
#page.get_text("text", flags=8)
#page.get_text_blocks()

page_display = page.get_displaylist()
dictionary_elements = page_display.get_textpage().extractDICT()
for block in dictionary_elements['blocks']:
    for line in block['lines']:
        #print(block)
        line_text = ''
        #print(line)
        #break
        for span in line['spans']:
            #if span['font']..lower()
            line_text += ' ' + span['text']
            #print(line_text)
        #print("NEW LINE")


#{'number': 0, 'type': 0, 'bbox': (166.8000030517578, 26.6500244140625, 399.86077880859375, 35.6500244140625), 'lines': [{'spans': [{'size': 8.954887390136719, 'flags': 4, 'font': 'Times-Roman', 'color': 0, 'ascender': 0.75, 'descender': -0.25, 'text': 'NOUVELLE FLORE ', 'origin': (166.8000030517578, 33.4000244140625), 'bbox': (166.8000030517578, 26.6500244140625, 247.62559509277344, 35.6500244140625)}], 'wmode': 0, 'dir': (1.0, 0.0), 'bbox': (166.8000030517578, 26.6500244140625, 247.62559509277344, 35.6500244140625)}, {'spans': [{'size': 8.954887390136719, 'flags': 4, 'font': 'Times-Roman', 'color': 0, 'ascender': 0.75, 'descender': -0.25, 'text': '563', 'origin': (384.6168518066406, 33.4000244140625), 'bbox': (384.6168518066406, 26.6500244140625, 399.86077880859375, 35.6500244140625)}], 'wmode': 0, 'dir': (1.0, 0.0), 'bbox': (384.6168518066406, 26.6500244140625, 399.86077880859375, 35.6500244140625)}]}
#{'spans': [{'size': 8.954887390136719, 'flags': 4, 'font': 'Times-Roman', 'color': 0, 'ascender': 0.75, 'descender': -0.25, 'text': 'NOUVELLE FLORE ', 'origin': (166.8000030517578, 33.4000244140625), 'bbox': (166.8000030517578, 26.6500244140625, 247.62559509277344, 35.6500244140625)}], 'wmode': 0, 'dir': (1.0, 0.0), 'bbox': (166.8000030517578, 26.6500244140625, 247.62559509277344, 35.6500244140625)}

In [42]:
page.get_text("words");
for b in page.get_text("dict")["blocks"]:
    if 'lines' in b:
        b['lines']

In [43]:
#page = doc.getPageText(2, output="json")
page = doc.load_page(page_num-4)

blocks = page.get_text("dict")["blocks"]
for b in blocks:
    if "lines" in b:
        for l in b["lines"]:
            for span in b["lines"]:
                for word in span["spans"][0]["text"].split():
                    print("word:", word, "\t\t\t flags:", span["spans"][0]["flags"], "\t\t\t font:", span["spans"][0]["font"])

word: INDEX 			 flags: 4 			 font: Times-Roman
word: SPECIERUM 			 flags: 4 			 font: Times-Roman
word: A 			 flags: 4 			 font: Times-Roman
word: Acacia 			 flags: 4 			 font: Times-Roman
word: albida 			 flags: 4 			 font: Times-Roman
word: Del. 			 flags: 4 			 font: Times-Roman
word: cyanophylla 			 flags: 6 			 font: Times-Italic
word: farnesiana 			 flags: 4 			 font: Times-Roman
word: seyal 			 flags: 6 			 font: Times-Italic
word: spirocarpa 			 flags: 6 			 font: Times-Italic
word: seyal 			 flags: 6 			 font: Times-Italic
word: spirocarpa 			 flags: 6 			 font: Times-Italic
word: tortilis 			 flags: 6 			 font: Times-Italic
word: Acer 			 flags: 4 			 font: Times-Roman
word: hermoneum 			 flags: 4 			 font: Times-Roman
word: kyrcanum 			 flags: 6 			 font: Times-Italic
word: monspessulanum 			 flags: 6 			 font: Times-Italic
word: syriacum 			 flags: 4 			 font: Times-Roman
word: Boiss. 			 flags: 4 			 font: Times-Roman
word: et 			 flags: 4 			 font: Times-Roman
word: tauri

# Finding indentations associated with genus, epithet, infra

In [44]:
types = ['genus', 'epithet', 'infra', 'author', 'misc.']
def n_leftmost_indent(df, n):
    """return a tuple with at most 3 elements each element itself is a tuple containing indent group, mean, group len"""
    indent_groups = [(g, df[(df['indent_group'] == g) & (df['word_no'] == 0)]['x0'].mean(), len(df[(df['indent_group'] == g) & (df['word_no'] == 0)]['x0'])) for g in df['indent_group'].unique()]
    indent_groups.sort(key = lambda x : x[1])
    #print(indent_groups[:n])
    return indent_groups[:n]

In [45]:
def get_genusEpithetInfra_indent(col_df):
    leftmost_3_indents = n_leftmost_indent(col_df, 3) #for vol1 only 2 indentations will be given 
    min_gap = 0
    max_gap = 75 #error is 30 -- less than 50% of max gap (which will be ignored for now)

    # possibly not specific enough
    # first identifying indent based don distance from one another only
    """if len(leftmost_3_indents) == 3:
        if leftmost_3_indents[0][1] < max_gap:
            leftmost_3_indents = leftmost_3_indents[1:]
        elif ((leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) > max_gap or \
            (leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) < min_gap): #comparing first two (if satisfied last two will be checked in next if block)
            leftmost_3_indents = [max(leftmost_3_indents[1:], key = lambda x : x[2])] + [leftmost_3_indents[2]]
        elif (leftmost_3_indents[2][1] - leftmost_3_indents[1][1]) > max_gap or \
            (leftmost_3_indents[2][1] - leftmost_3_indents[1][1]) < min_gap: #comparing last two
            leftmost_3_indents = [leftmost_3_indents[0]] + [max(leftmost_3_indents[1:], key = lambda x : x[2])]

    if len(leftmost_3_indents) == 2:
        if leftmost_3_indents[0][1] < max_gap:
            leftmost_3_indents = leftmost_3_indents[1]
        elif (leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) > max_gap or (leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) < min_gap:
            leftmost_3_indents = [max(leftmost_3_indents, key = lambda x : x[2])]"""

    has_genus, has_epithet, has_infra = False, False, False
    genus_indent, epithet_indent, infra_indent = -1, -1, -1
    if len(leftmost_3_indents) == 3 and type(leftmost_3_indents) == type([1,2,3]):
        has_genus, has_epithet, has_infra = True, True, True
        #print("leftmost 3:", leftmost_3_indents)
        genus_indent, epithet_indent, infra_indent = [el[0] for el in leftmost_3_indents]
    elif len(leftmost_3_indents) == 2:
        if col_df[col_df['indent_group'] == leftmost_3_indents[1][0]]['word'].apply(is_infra).any():
            has_genus, has_epithet, has_infra = False, True, True
            epithet_indent, infra_indent = [el[0] for el in leftmost_3_indents]
        else:
            has_genus, has_epithet, has_infra = True, True, False
            genus_indent, epithet_indent = [el[0] for el in leftmost_3_indents]
    elif len(leftmost_3_indents) == 1 or type(leftmost_3_indents) == type((1,2,3)): 
        if type(leftmost_3_indents) == type((1,2,3)):
            leftmost_3_indents = [leftmost_3_indents]
        has_genus, has_epithet, has_infra = False, True, False
        epithet_indent = leftmost_3_indents[0][0]

    return genus_indent, epithet_indent, infra_indent, leftmost_3_indents

# Processing column dataframes


In [46]:
def process_col(col_df, genus, epithet, draw_genus, draw_epithet, draw_infra = np.NaN):
    genus_indent, epithet_indent, infra_indent, indent_3_left = get_genusEpithetInfra_indent(col_df)
    #print(genus_indent, epithet_indent, infra_indent, indent_3_left)
    
    blocks = col_df['block_no'].unique()
    start_word_cond = -1 
    author = ''
    #draw_infra = np.NaN

    process_hybrid = False
    process_infra = False
    hybrid_pre_str = ''

    col_df = col_df.copy()
    for index, row in col_df.iterrows():
        b, l, w = row['block_no'], row['line_no'], row['word_no']
        word, indent_group = row['word'], row['indent_group']
        row_cond = (col_df['line_no'] == l) & (col_df['block_no'] == b) & (col_df['word_no'] == w) 
        
        if w == 0 or process_hybrid: 
            if is_hybrid(word):
                process_hybrid = True
                misc = word
                author = ''
                hybrid_pre_str = 'hybrid-'
            
            else:
                if process_hybrid: 
                    #col_df.loc[start_word_cond, 'misc.'] = f"hybrid ({misc})"
                    process_hybrid = False
                    hybrid_pre_str = 'hybrid-'
                    #print("hi")
                else: 
                    hybrid_pre_str = ''

                prev_start_cond = start_word_cond
                start_word_cond = row_cond
                process_hybrid = False
                process_infra = False
                
                if indent_group == genus_indent and not ''.join(e for e in word if e.isalpha()).isupper():
                    process_infra = False
                    genus = word
                    draw_genus = genus
                    epithet = ''
                    draw_epithet = ''
                    author = ''
                    misc = ''
                    infra = ''
                    col_df.loc[start_word_cond, 'genus'] = genus
                    col_df.loc[start_word_cond, 'taxon rank'] = hybrid_pre_str  + 'genus'
                    if not is_genus(word):
                        col_df.loc[row_cond, 'error_check'] = True
                    col_df.loc[row_cond, 'draw_genus'] = draw_genus
                    col_df.loc[row_cond, 'author'] = ''

                elif indent_group == epithet_indent and not ''.join(e for e in word if e.isalpha()).isupper():
                    process_infra = False
                    epithet = word
                    author = ''
                    col_df.loc[row_cond, 'genus'] = genus
                    col_df.loc[row_cond, 'epithet'] = epithet
                    col_df.loc[row_cond, 'taxon rank'] = hybrid_pre_str  + 'species'
                    if not is_epithet(word):
                        col_df.loc[row_cond, 'error_check'] = True
                    draw_epithet = str(genus) + '_' + str(epithet) +'_' + str(b) + '_' + str(l)
                    col_df.loc[row_cond, 'draw_genus'] = draw_genus
                    col_df.loc[row_cond, 'draw_epithet'] = draw_epithet
                    col_df.loc[row_cond, 'author'] = ''

                elif indent_group == infra_indent and is_infra:
                    #col_df.loc[row_cond, 'misc.'] = word
                    process_infra = True
                    misc = word
                    author = ''
                    if not (is_infra(word) or is_hybrid(word)):
                        col_df.loc[start_word_cond, 'error_check'] = True
                        #possibly should add to author of previous line here? 
                        #print("possible author?", word)
                    #print("hi")

        elif w == 1 and process_infra and (is_infra(misc) or len(''.join(e for e in misc if e.isalpha())) <2) and (misc != "et"):
            infra = word 
            start_word_cond = row_cond
            #print(updated)
            #print(b, l, w)
            #col_df.loc[row_cond, 'misc.'] = misc
            col_df.loc[start_word_cond, 'taxon rank'] = hybrid_pre_str + misc
            col_df.loc[start_word_cond, 'genus'] = genus
            col_df.loc[start_word_cond, 'epithet'] = epithet
            col_df.loc[start_word_cond, 'infra'] = infra
            col_df.loc[start_word_cond, 'author'] = ''
            draw_infra = str(infra) + '_'+str(b)+'_'+str(l)
            process_infra = False
            col_df.loc[start_word_cond, 'draw_genus'] = draw_genus
            col_df.loc[start_word_cond, 'draw_epithet'] = draw_epithet
            col_df.loc[start_word_cond, 'draw_infra'] = draw_infra
        
        elif process_infra:
            #print(misc, word)
            if misc != "":
                curr_author_part = misc + ' ' + word + ' '
                col_df.loc[start_word_cond, 'draw_author'] = 'author_'+str(b)+'_'+str(l)
            else:
                curr_author_part = word + ' '
            col_df.loc[prev_start_cond, 'author'] += curr_author_part
            col_df.loc[row_cond, 'draw_author'] = 'author_'+str(b)+'_'+str(l)
            col_df.loc[row_cond, 'draw_genus'] = draw_genus

        elif (type(genus) == type("STR") and genus != '') or (type(epithet) == type("STR") and epithet != ''):
            curr_author_part = word + ' '
            col_df.loc[start_word_cond, 'author'] += curr_author_part
            col_df.loc[row_cond, 'draw_author'] = 'author_'+str(b)+'_'+str(l)
            col_df.loc[row_cond, 'draw_genus'] = draw_genus

    #Last author
    """if author != '':
        col_df.loc[start_word_cond, 'author'] = author"""
                    

    return col_df, genus, epithet, draw_genus, draw_epithet


# Run PreProcessing

In [47]:
#preprocessing
genus = np.NaN
df_dict = {}
pruned_dict = {}

for page_num in tqdm(index):
    page_df, pruned_df, indent_group = preprocessing(page_num)
    df_dict[page_num] = page_df
    pruned_dict[page_num] = pruned_df

genus = np.NaN
epithet = np.NaN
draw_genus = np.NaN
draw_epithet = np.NaN
result_ims_valid_words = []
df_list = []

for page_num in tqdm(index):
    #page_num = index[-1]
    #process the pre-processed dfs
    page_df = df_dict[page_num]
    
    #for drawing
    pix_map = doc.get_page_pixmap(page_num,matrix=mat)
    image = Image.open(io.BytesIO(pix_map.tobytes()))
    draw = ImageDraw.Draw(image)
    
    plot_valid_words(page_df, draw, color = '#660044', w = 2)
    result_ims_valid_words.append(image)
    
    #break 
#result_ims_valid_words[0].save(OUTPUT_PATH + "preprocessed/" + 'valid_words' + TAIL_STR + '.pdf',save_all=True, append_images=result_ims[1:])

100%|██████████| 22/22 [00:09<00:00,  2.34it/s]


# Final Results + SAVE

In [48]:
#Setting up files and directories for saving the results
SCRIPT_NAME = "reverted_index_Vol2_synonyms_pruned.ipynb"
SCRIPT_OUTPUT_PATH = "../output/index/" + SCRIPT_NAME + "/"
DATE_STR = datetime.now().strftime("%Y_%m_%d") 
TIME_STR = datetime.now().strftime("%H%M")
QUICK_FIX = False
TAIL_STR = ''

if QUICK_FIX:
    OUTPUT_PATH = SCRIPT_OUTPUT_PATH + DATE_STR + "/QuickFix/" 
    #TAIL_STR = '_' + DATE_STR + '_' + TIME_STR
else:
    OUTPUT_PATH = SCRIPT_OUTPUT_PATH + DATE_STR + "/" + TIME_STR + "/"

try:
    os.makedirs(OUTPUT_PATH)
except FileExistsError:
    # directory already exists
    pass

try:
    os.makedirs(OUTPUT_PATH + "preprocessed/")
except FileExistsError:
    # directory already exists
    pass

try:
    os.makedirs(OUTPUT_PATH + 'raw/')
except FileExistsError:
    # directory already exists
    pass

In [49]:
pre_processed_df = pd.concat([df_dict[k] for k in df_dict], axis = 0)
result_ims_valid_words[0].save(OUTPUT_PATH + "preprocessed/" + 'valid_words' + TAIL_STR + '.pdf',save_all=True, append_images=result_ims_valid_words[1:])
pre_processed_df.to_html(OUTPUT_PATH + "preprocessed/" + 'vol2_preprocessed_index' + TAIL_STR + '.html')
pre_processed_df.to_csv(OUTPUT_PATH + "preprocessed/" + 'vol2_preprocessed_index' + TAIL_STR + '.csv')

In [20]:
genus = np.NaN
epithet = np.NaN
draw_genus = np.NaN
draw_epithet = np.NaN
result_ims = []
df_list = []

for page_num in tqdm(index):
    #if page_num == index[-2]:
    #    break
    #page_num = index[-1]
    #process the pre-processed dfs
    page_df = df_dict[page_num]
    
    #for drawing
    pix_map = doc.get_page_pixmap(page_num,matrix=mat)
    image = Image.open(io.BytesIO(pix_map.tobytes()))
    draw = ImageDraw.Draw(image)

    #processing each column
    for c in page_df['col_no'].unique():
        col_df = page_df[page_df['col_no'] == c]
        col_df, genus, epithet, draw_genus, draw_epithet = process_col(col_df, genus, epithet, draw_genus, draw_epithet)
        df_list.append(col_df)

        #drawing boxes in each column
        plot_genus_blocks(col_df, draw)
        plot_epithet_blocks(col_df, draw)
        plot_author_blocks(col_df, draw)
        plot_infra_blocks(col_df, draw)

    result_ims.append(image)

#TIME_STR = datetime.now().strftime("%Y_%m_%d-%I_%M_%p")
result_ims[0].save(OUTPUT_PATH + 'vol2_index_ROI.pdf',save_all=True, append_images=result_ims[1:])

pre_processed_df = pd.concat([df_dict[k] for k in df_dict], axis = 0)
result_ims_valid_words[0].save(OUTPUT_PATH + "preprocessed/" + 'valid_words' + TAIL_STR + '.pdf',save_all=True, append_images=result_ims_valid_words[1:])
pre_processed_df.to_html(OUTPUT_PATH + "preprocessed/" + 'vol2_preprocessed_index' + TAIL_STR + '.html')
pre_processed_df.to_csv(OUTPUT_PATH + "preprocessed/" + 'vol2_preprocessed_index' + TAIL_STR + '.csv')

df = pd.concat(df_list, axis = 0)
df.to_html(OUTPUT_PATH + 'raw/' + 'vol2_index' + TAIL_STR + '.html')
df.to_csv(OUTPUT_PATH + 'raw/' + 'vol2_index' + TAIL_STR + '.csv', index = False)

pruned = df[(~df['genus'].isnull())]
pruned = pruned[["page_num", "genus", "epithet", "infra" ,"author", "taxon rank"]]
pruned.to_csv(OUTPUT_PATH + 'vol2_index_pruned' + TAIL_STR + '.csv', index = False)
pruned.to_html(OUTPUT_PATH + 'vol2_index_pruned' + TAIL_STR + '.html')

 18%|█▊        | 4/22 [00:07<00:34,  1.93s/it]


KeyError: -1

# YEET?

In [29]:
import fitz
import pprint, re

def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

class Face:
    def __init__(self, font, size, color, flags=0):
        self.font = font
        self.size = size
        self.color = color
        self.flags = flags
        self.style = flags_decomposer(self.flags)
    def __str__(self):
        return f"Font: '{self.font}' ({self.style}), size {self.size:g}, color " + ("#%06x" % self.color)

In [30]:
def preprocessing(page_num, indent_err = 30):
    
    #initiate dataframe
    page_df = pd.DataFrame(pages[page_num].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
    
    #add page number to dataframe
    page_df['page_num'] = np.array([page_num]*page_df.shape[0])
    #initiate all columns that will be added
    page_df['genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['taxon rank'] = np.array([np.NaN]*page_df.shape[0])
    page_df['error_check'] = np.array([np.NaN]*page_df.shape[0])
    #updating coordinates to represent target DPI
    page_df['x0'], page_df['y0'], page_df['x1'], page_df['y1'] = page_df['in_x0']*TARGET_DPI/ 72, page_df['in_y0']*TARGET_DPI/ 72, page_df['in_x1']*TARGET_DPI/ 72, page_df['in_y1']*TARGET_DPI/ 72
    #get x corner coordinates 
    x_min = page_df['x0'].min()
    x_max = page_df['x1'].max()

    y_max = page_df['y1'].max()

    #Remove the extra flore - 18 at page 545
    if page_num == index[4]:
        page_df = page_df[~((page_df["word"] == 'Flore') & (page_df['y1'] == y_max))]
    #invalid words dataframe -- for error checking
    pruned_words_df = page_df[~page_df["word"].apply(valid)].reset_index()

    #use this face thing to get the words  styles? 

    #prune out invalid words (based on function valid)
    #page_df = page_df[page_df["word"].apply(valid)].reset_index()
    
    indent_groups = []
    blocks = page_df['block_no'].unique()
    for b in blocks:
        lines = page_df[page_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            #reset word_no values (useful for cases where word that was originally at 0th index was pruned out)
            cond = (page_df['line_no'] == l) & (page_df['block_no'] == b)
            num_words = len(page_df[cond]['word_no'])
            page_df.loc[cond, 'word_no'] = np.arange(num_words).astype(int) #this is slowww
            #set column number (0 or 1)
            x_0 = page_df[cond]['x0'].min()
            #THIS DOESN'T WORK AAAA -- issue was with line no thing
            if not np.isnan(x_0):
                page_df.loc[cond, 'col_no'] = np.array([int(x_0 > ((x_min + x_max) / 2))]*num_words).astype(int)

                #initiate indent groups -- only first word should get an indent_group value 
                new_group = True
                for g_i in range(len(indent_groups)):
                    g = indent_groups[g_i]
                    g_arr = np.array(g)
                    if x_0 <= np.mean(g_arr) + indent_err and x_0 >= np.mean(g_arr) - indent_err:
                        g.append(x_0)
                        new_group = False
                        page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)
                if new_group:
                    indent_groups.append([x_0])
                    g_i = len(indent_groups) - 1
                    page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)

    #print("indent groups:", indent_groups)
    #return updated page_df, pruned_words_df, indent groups
    return page_df.reset_index(), pruned_words_df, indent_groups

#https://stackoverflow.com/questions/53468558/adding-image-to-pandas-dataframe